In [ ]:
!pip install pandas
!pip install matplotlib
!pip install catboost
!pip install scikit-learn  

IMPORTS

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# import seaborn as sns
from catboost import CatBoostRegressor, Pool, cv 
from sklearn.model_selection import train_test_split
from catboost.utils import get_roc_curve
import sklearn
from sklearn import metrics


READ DATASETS

In [42]:
df = pd.read_csv('datasets/out_dataset_3_class_porog_60.csv')
y = df.pop('price_doc')
del df['price_group']
cat_features = [i for i, column in enumerate(df.columns) if df[column].dtypes == object]
X = df


SPLIT TRAIN AND TEST DATA

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=60)

In [ ]:
X_train

In [45]:
train_pool = Pool(X_train, y_train, cat_features=cat_features)
test_pool = Pool(X_test, y_test, cat_features=cat_features)

INITIALIZING REGRESSOR

In [48]:
grid = {
    'learning_rate': [0.09, 0.09],
    'depth': [6,7,6],
    'l2_leaf_reg': [7,1],
    'grow_policy': ['SymmetricTree']
    
}
model = CatBoostRegressor(
    iterations=7500,  
    cat_features=cat_features, 
    loss_function='RMSE', 
    early_stopping_rounds=20,
    learning_rate=grid['learning_rate'][0], 
    l2_leaf_reg=grid['l2_leaf_reg'][0], 
    grow_policy=grid['grow_policy'][0],
    depth=grid['depth'][0]
    
    )

FIT REGRESSOR

In [ ]:
# model.randomized_search(grid, X=train_pool,plot = True)
model.fit(
          train_pool, 
          plot=True,
          verbose=False  
          )

TEST MODEL

In [51]:

predictions = model.predict(Pool(X_test, y_test, cat_features=cat_features))
predictions - y_train

METRICS (ACCURACY)

In [ ]:
model.score(train_pool), model.score(test_pool)

SAVE MODEL

In [23]:
model.save_model("models/catboost-model-1-acc-64-097-2-new_ver")